In [1]:
import pandas as pd
import requests
import random
import time
import os
import openpyxl
from bs4 import BeautifulSoup

def get_url(stk_code,pages):
    '''
    获取东方财富网股吧链接列表
    code是指公司代码
    page是值爬取页数
    '''
    code = stk_code.split('.')[0]
    url_list = []
    for page in range(1,pages+1):
        url = f"http://guba.eastmoney.com/list,{code},1,f_{page}.html"
        url_list.append(url)
        
    return url_list

def get_news(url_list,stk_code):
    '''
    获取东方财富网新闻列表至本地xls
    url_list是指链接列表
    '''
    code = stk_code.split('.')[0]
    code_suffix = stk_code.split('.')[1]
    headers = {
        # 'User-Agent': UserAgent(verify_ssl=False).random,
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
        'cookie': 'qgqp_b_id=b90ae58b50ba4b8868c1988b80823e4c; st_si=62771611099798; st_asi=delete; st_pvi=50748890314615; st_sp=2022-10-24 11:22:01; st_inirUrl=http://guba.eastmoney.com/list,hk01810,1,f_1.html; st_sn=2; st_psi=20221024112234183-117001300541-9748756866'
    }
    
    # 保存爬取内容
    outwb = openpyxl.Workbook() # 打开一个将写的文件
    outws = outwb.create_sheet(index=0) # 在将写的文件创建sheet
    outws.cell(row = 1, column = 1, value = "read")
    outws.cell(row = 1, column = 2, value = "comment")
    outws.cell(row = 1, column = 3, value = "title")
    outws.cell(row = 1, column = 4, value = "author")
    outws.cell(row = 1, column = 5, value = "renew")
    outws.cell(row = 1, column = 6, value = "link")
    index = 2
    
    for i in range(len(url_list)):
        url = url_list[i]
        res = requests.get(url,headers = headers)
        res.encoding = res.apparent_encoding
        html = res.text
        soup = BeautifulSoup(html,"html.parser")
        read_list = soup.select(".l1.a1")[1:]
        comment_list = soup.select(".l2.a2")[1:]
        title_list = soup.select(".l3.a3")[1:]
        author_list = soup.select(".l4.a4")[1:]
        renew_list = soup.select(".l5.a5")[1:]
        for k in range(len(title_list)):
            outws.cell(row = index, column = 1, value = str(read_list[k].text.strip()))
            outws.cell(row = index, column = 2, value = str(comment_list[k].text.strip()))
            outws.cell(row = index, column = 3, value = str(title_list[k].select('a')[0]["title"]))
            outws.cell(row = index, column = 4, value = str(author_list[k].text.strip()))
            outws.cell(row = index, column = 5, value = str(renew_list[k].text.strip()))
            outws.cell(row = index, column = 6, value = str(title_list[k].select('a')[0]["href"]))                                                
            index += 1
            # print(title_list[k].select('a')[0]["title"],renew_list[k].text.strip())
        time.sleep(random.uniform(3,4))
            
    outwb.save(f"./news_ann/news/tmp/{code}.{code_suffix}.xlsx")
    df = pd.read_excel(f"./news_ann/news/tmp/{code}.{code_suffix}.xlsx")
    return df

# code = "hk01810"
# code = "603355"
# pages = 5
# url_list = get_url(code,pages)
# get_news(url_list,code)
# print("运行完成")    

stk_code_list = ['300587.SZ',
'603185.SH',
'600674.SH',
'002203.SZ',
'601127.SH',
'002597.SZ',
'603348.SH',
'300487.SZ',
'603876.SH',
'601677.SH',
'300655.SZ',
'002078.SZ',
'002738.SZ', #2022-10-14新增4个对子
'002812.SZ',
'603596.SH',
'300332.SZ',]

date = '20221031'

formatted_date = date[4:6]+'-'+date[6:]
df_full_res = []
for stk_code in stk_code_list:
    # code = stk_code
    pages = 2
    url_list = get_url(stk_code,pages)
    df_tmp = get_news(url_list,stk_code)
    df_tmp['code'] = [stk_code]*len(df_tmp)
    df_full_res.append(df_tmp)

    print(f"{stk_code} 运行完成")    


300587.SZ 运行完成
603185.SH 运行完成
600674.SH 运行完成
002203.SZ 运行完成
601127.SH 运行完成
002597.SZ 运行完成
603348.SH 运行完成
300487.SZ 运行完成
603876.SH 运行完成
601677.SH 运行完成
300655.SZ 运行完成
002078.SZ 运行完成
002738.SZ 运行完成
002812.SZ 运行完成
603596.SH 运行完成
300332.SZ 运行完成


In [2]:
df_single_day_res = []
for df_tmp in df_full_res:
    
    df_tmp['date'] = df_tmp['renew'].apply(lambda x : x.split(' ')[0])
    df_tmp = df_tmp[df_tmp['date']==formatted_date]
    if len(df_tmp)!=0:
        df_tmp = df_tmp[['code','date','title','author']]
        df_single_day_res.append(df_tmp)

In [3]:
if len(df_single_day_res)!=0:
    df_total = pd.concat(df_single_day_res)    
    df_total.to_excel(f'./news_ann/news/{date}_news.xlsx')
else:
    df_total = pd.DataFrame([])
    df_total.to_excel(f'./news_ann/news/{date}_news.xlsx')

# df_total = pd.concat(df_single_day_res)
# df_total.to_excel(f'./news_ann/news/{date}_news.xlsx')
df_total

,code,date,title,author
2,300587.SZ,10-31,天铁股份：融资净买入1734.23万元，融资余额6852.89万元（10-28）,天铁股份资讯
2,603185.SH,10-31,上机数控：“上22转债”赎回登记日为11月15日,上机数控资讯
2,600674.SH,10-31,川财证券维持川投能源增持评级 预计2022年净利润同比增长11.76%,公司研报提示
3,600674.SH,10-31,川财证券给予川投能源增持评级,川投能源资讯
2,002203.SZ,10-31,东吴证券给予海亮股份买入评级 2022Q3业绩点评：汇兑损益影响扣非利润 铜箔进展较快,海亮股份资讯
...,...,...,...,...
6,603596.SH,10-31,东方证券维持伯特利买入评级 目标价87.72元,公司研报提示
7,603596.SH,10-31,东亚前海证券维持伯特利推荐评级 预计2022年净利润同比增长23.59%,公司研报提示
8,603596.SH,10-31,东亚前海证券给予伯特利推荐评级 多产品线营收稳步提升 有力带动公司三季度业绩创新高,伯特利资讯
9,603596.SH,10-31,光大证券维持伯特利买入评级 目标价98.96元,公司研报提示


In [4]:
# date = '20221101'
# formatted_date = date[4:6]+'-'+date[6:]
# formatted_date

In [5]:
url_list

['http://guba.eastmoney.com/list,300332,1,f_1.html',
 'http://guba.eastmoney.com/list,300332,1,f_2.html']